In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pandas_ta as ta
import os


# Specify the path to your CSV file
csv_file = 'C:\\Users\\prasa\\Downloads\\ind_stock.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file)
df = df.drop_duplicates()
def get_stock_data(ticker, interval, start, end):
#     stock = yf.Ticker(ticker)
#     data = stock.history(interval=interval, start=start)
    data = yf.download(ticker, period= interval, start=start , end = datetime.now())
    return data

def is_bullish_engulfing(data):
    bullish_engulfing = []
    for i in range(1, len(data)):
        prev_open = data['Open'].iloc[i-1]
        prev_close = data['Close'].iloc[i-1]
        curr_open = data['Open'].iloc[i]
        curr_close = data['Close'].iloc[i]
        
        if curr_open <= prev_close and curr_open < prev_open and curr_close > prev_open and curr_close > prev_close:
            bullish_engulfing.append(True)
        else:
            bullish_engulfing.append(False)
    # The first entry cannot be determined, so we prepend a False
    bullish_engulfing.insert(0, False)
    data['Bullish Engulfing'] = bullish_engulfing
    return data

def is_bullish_thrusting(data):
    bullish_thrusting = []
    bullish_tweezer_bottom = []
    for i in range(1, len(data)):
        prev_open = data['Open'].iloc[i-1]
        prev_close = data['Close'].iloc[i-1]
        curr_open = data['Open'].iloc[i]
        curr_close = data['Close'].iloc[i]
        last_candle_movement = (((prev_close - prev_open) / prev_open) * 100)
        if last_candle_movement < 0 :
            if curr_open > prev_close  and curr_close > prev_open:
                bullish_thrusting.append(True)
            else:
                bullish_thrusting.append(False)
        else :
            bullish_thrusting.append(False)
            
        prev_open = data['Open'].iloc[i-1]
        prev_close = data['Close'].iloc[i-1]
        prev_low = data['Low'].iloc[i-1]
        curr_open = data['Open'].iloc[i]
        curr_close = data['Close'].iloc[i]
        curr_low = data['Low'].iloc[i]
        
        if prev_low == curr_low and prev_close < prev_open and curr_close > curr_open:
            bullish_tweezer_bottom.append(True)
        else:
            bullish_tweezer_bottom.append(False)
    # The first entry cannot be determined, so we prepend a False
    bullish_thrusting.insert(0, False)
    data['Bullish Thrusting'] = bullish_thrusting
    bullish_tweezer_bottom.insert(0, False)
    data['Bullish Tweezer Bottom'] = bullish_tweezer_bottom
    return data

def add_percentage_change(data):
    past5days_change = [None] * len(data)
    past10days_change = [None] * len(data)
    past15days_change = [None] * len(data)
    BE_condition = [False] * len(data)
    BT_condition = [False] * len(data)
    Max_gain_on  = [5] * len(data)
    Max_gain  = [None] * len(data)
    SL_Hit_on  = [None] * len(data)
    SL_rs = 1000
    Take_trade = [False] * len(data)
    for i in range(0, len(data)):
        if data['Bullish Thrusting'].iloc[i] == True or data['Bullish Engulfing'].iloc[i] == True or data['Bullish Tweezer Bottom'].iloc[i] == True :
            Take_trade[i] = True
    data['Take Trade'] = Take_trade
    for i in range(5, len(data)):
        open_5_days_ago = data['Open'].iloc[i-5]
        current_open = data['Open'].iloc[i]
        change_pct = ((current_open - open_5_days_ago) / open_5_days_ago) * 100
        past5days_change[i] = change_pct
    for i in range(10, len(data)):
        open_10_days_ago = data['Open'].iloc[i-10]
        current_open = data['Open'].iloc[i]
        change_pct = ((current_open - open_10_days_ago) / open_10_days_ago) * 100
        past10days_change[i] = change_pct
    for i in range(15, len(data)):
        open_15_days_ago = data['Open'].iloc[i-15]
        current_open = data['Open'].iloc[i]
        change_pct = ((current_open - open_15_days_ago) / open_15_days_ago) * 100
        past15days_change[i] = change_pct 
    for i in range(15, len(data)) :
        try :
            if (data['CRS SMA'].iloc[i] <= data['CRS'].iloc[i]) and int(data['ATR%'].iloc[i]) >= 3 and int(past5days_change[i]) <=0 and ((int(past5days_change[i]) >= int(past10days_change[i])) and  (int(past10days_change[i]) >= int(past15days_change[i]))) :
                BE_condition[i] = True
            if (data['CRS SMA'].iloc[i] <= data['CRS'].iloc[i]) and int(data['ATR%'].iloc[i]) >= 3 and int(past5days_change[i]) <=-4 and ((int(past5days_change[i]) >= int(past10days_change[i])) and  (int(past10days_change[i]) >= int(past15days_change[i]))) :
                BT_condition[i] = True
        except :
            pass
    data['-5 Day Change %'] = past5days_change
    data['-10 Day Change %'] = past10days_change
    data['-15 Day Change %'] = past15days_change
    data['BE Selection'] = BE_condition
    data['BT Selection'] = BT_condition
    exist_days = 5
    past_days = 15
    Quan = [None] * len(data)
    Fund = [None] * len(data)
    SL = [False] * len(data)
    for i in range(5, len(data)):
        sl = data['Low'].iloc[i]
        current_close = data['Close'].iloc[i]
        sl_price = max(current_close - sl,0.1)
        try :
            quantity = max(round(SL_rs/sl_price),1)
        except :
            quantity = 1
        trade_price = quantity * current_close
        Quan[i] = quantity
        Fund[i] = trade_price
        SL[i] = sl_price
    data[f'Quantity'] = Quan
    data[f'Fund'] = Fund
    data[f'SL'] = SL
            
    for j in range(5,exist_days+1):
        futuredays_change = [None] * len(data)
        futuredays2_change = [None] * len(data)
        futuredays_close = [None] * len(data)
        sl_hit = [False] * len(data)
        PNL = [None] * len(data)
        PNL2 = [None] * len(data)
        for i in range(0, len(data)-j):
            open_5_days_later = data['Close'].iloc[i+j]
            current_close = data['Close'].iloc[i]
            sl = data['Low'].iloc[i]
            change_pct = ((open_5_days_later - current_close ) / current_close) * 100
            futuredays_change[i] = change_pct 
            if Max_gain[i] == None :
                Max_gain[i] = change_pct
                Max_gain_on[i] = j
                
            if Max_gain[i] < change_pct:
                Max_gain[i] = change_pct
                Max_gain_on[i] = j
                
            sl_price = max(current_close - sl,0.1)
            try :
                quantity = max(round(SL_rs/sl_price),1)
            except :
                quantity = 1
            
            trade_price = quantity * current_close
            PNL[i] = (open_5_days_later - current_close ) * quantity
            Quan[i] = quantity
            Fund[i] = trade_price
            SL[i] = sl_price
        for i in range(0, len(data)-j):
            open_5_days_later = data['Close'].iloc[i+j]
            next_open = data['Open'].iloc[i+1]
            change_pct = ((open_5_days_later - next_open ) / next_open) * 100
            futuredays2_change[i] = change_pct
        for i in range(0, len(data)-j):
            open_5_days_later = data['Close'].iloc[i+j]
            futuredays_close[i] = open_5_days_later
            for k in range(1, j+1):
                sl = data['Low'].iloc[i]
                current_close = data['Close'].iloc[i]
                if sl >= data['Low'].iloc[i+k]:
                    sl_hit[i] = True
                    PNL[i] = -(SL[i] * Quan[i])
                    if SL_Hit_on[i] == None:
                        SL_Hit_on[i] = k
                
        data[f'cdc {j} Day After c %'] = futuredays_change
        data[f'cdc {j} Day After c PNL'] = PNL
        #data[f'ndo {j} Day After c %'] = futuredays2_change
        #data[f'{j}-Day After exit'] = futuredays_close
        data[f'SL Hit {j} Day'] = sl_hit
    data[f'Max_gain_on_day'] = Max_gain_on
    data[f'Max_gain'] = Max_gain
    data[f'SL_HIT_On'] = SL_Hit_on
    return data

def add_comparative_rsi(data, benchmark_data, length=100, lookbackMATrend=3):
    baseSymbol = data['Close']
    comparativeSymbol = benchmark_data['Close']
    
    crs = baseSymbol / comparativeSymbol
    crsSma = ta.sma(baseSymbol / comparativeSymbol, length)
    
    if crsSma is None:
        print("CRS SMA calculation returned None")
        return data
    
    if crsSma.isna().all():
        print("CRS SMA contains only NaN values")
        return data
    
    crsRising = crsSma > crsSma.shift(lookbackMATrend)
    crsFalling = crsSma < crsSma.shift(lookbackMATrend)
#     data['nifty close'] = comparativeSymbol
    data['CRS'] = crs
    data['CRS SMA'] = crsSma
    data['CRS Rising'] = crsRising
    data['CRS Falling'] = crsFalling
    
    return data

def add_atr_percentage(data):
    try :
        data['ATR%'] = round(((data['ATR'] / data['Close']) * 100), 2)
    except :
        data['ATR%'] = ((data['ATR'] / data['Close']) * 100)
    return data

combined_data = pd.DataFrame()
benchmark_ticker = '^NSEI'  # NIFTY50
end_date = datetime.now()
start_date = end_date - timedelta(days=500)
interval = "1d"
benchmark_data = get_stock_data(benchmark_ticker, interval, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

for index, row in df.iterrows():
    company_name = row['Company Name']
    industry = row['Industry']
    symbol = row['Symbol']
    series = row['Series']
    isin_code = row['ISIN Code']
    if series == 'BE':
        continue
    print(company_name)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=500)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    data = get_stock_data(symbol+'.NS', interval, start=start_date_str, end=end_date_str)
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=22)
    save_dir = f'D:\\Program Files\\StockData\\Daily\\{symbol}\\'
    os.makedirs(save_dir, exist_ok=True)
    file_name = f"Data_{interval}.csv"
    file_path = os.path.join(save_dir, file_name)
    data = is_bullish_engulfing(data)
    data = is_bullish_thrusting(data)
    data = add_atr_percentage(data)
    data = add_comparative_rsi(data, benchmark_data)
    data = add_percentage_change(data)
    print(data)
    data['Company Name'] = company_name
    data['Industry'] = industry
    data['Symbol'] = symbol
    data['Series'] = series
    data['ISIN Code'] = isin_code
    data.to_csv(file_path)
    combined_data = pd.concat([combined_data, data])

save_dir = 'D:\\Program Files\\StockData\\Combined_daily_data\\'
os.makedirs(save_dir, exist_ok=True)
combined_file_path = os.path.join(save_dir, f'Data_S1_{interval}.csv')
combined_data = combined_data.loc[combined_data['Take Trade'] == True]
combined_data = combined_data.loc[combined_data['BE Selection'] == True]
combined_data = combined_data.sort_index()
combined_data = combined_data.filter(like='2024-07', axis=0)
combined_data.to_csv(combined_file_path)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


360 ONE WAM Ltd.


[*********************100%%**********************]  1 of 1 completed


                   Open         High         Low        Close    Adj Close  \
Date                                                                         
2023-03-02   489.950012   489.950012  435.549988   441.049988   425.109314   
2023-03-03   443.950012   450.549988  440.100006   446.799988   430.651489   
2023-03-06   445.000000   455.000000  440.049988   446.600006   430.458771   
2023-03-08   446.000000   450.200012  440.000000   442.299988   426.314148   
2023-03-09   442.000000   448.899994  435.000000   440.100006   424.193634   
...                 ...          ...         ...          ...          ...   
2024-07-08  1034.949951  1034.949951  970.000000   981.400024   981.400024   
2024-07-09   984.000000  1011.799988  971.099976   996.849976   996.849976   
2024-07-10   997.650024  1022.250000  996.000000  1008.200012  1008.200012   
2024-07-11  1008.549988  1008.549988  961.250000   968.700012   968.700012   
2024-07-12   969.150024   969.650024  929.099976   936.549988   

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-03-02  287.000000  289.500000  275.000000  276.549988  273.485443   
2023-03-03  279.799988  283.250000  267.100006  269.350006  266.365234   
2023-03-06  272.000000  282.750000  272.000000  280.649994  277.540009   
2023-03-08  278.100006  279.799988  274.000000  277.700012  274.622742   
2023-03-09  278.950012  286.350006  277.549988  282.950012  279.814545   
...                ...         ...         ...         ...         ...   
2024-07-08  744.500000  744.599976  722.599976  734.349976  734.349976   
2024-07-09  735.500000  748.000000  724.549988  726.349976  726.349976   
2024-07-10  727.950012  738.000000  714.849976  734.700012  734.700012   
2024-07-11  737.849976  737.849976  714.450012  718.049988  718.049988   
2024-07-12  728.450012  788.000000  720.549988  769.700012  769.700012   

             Volume        ATR  Bulli


[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-03-02   54.750000   55.200001   54.349998   54.549999   54.549999   
2023-03-03   54.900002   55.400002   53.200001   53.650002   53.650002   
2023-03-06   54.000000   54.950001   53.700001   53.950001   53.950001   
2023-03-08   53.450001   55.299999   52.650002   54.799999   54.799999   
2023-03-09   55.099998   56.349998   53.900002   54.049999   54.049999   
...                ...         ...         ...         ...         ...   
2024-07-08  208.250000  210.000000  206.000000  207.929993  207.929993   
2024-07-09  207.800003  214.000000  206.610001  212.559998  212.559998   
2024-07-10  213.000000  213.389999  205.740005  211.369995  211.369995   
2024-07-11  211.470001  218.949997  211.369995  217.139999  217.139999   
2024-07-12  219.000000  223.250000  217.050003  222.470001  222.470001   

              Volume       ATR  Bulli


[*********************100%%**********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2023-03-02   474.000000   476.500000   465.200012   466.750000   457.982758   
2023-03-03   469.100006   474.649994   468.100006   472.950012   464.066345   
2023-03-06   475.149994   479.700012   474.100006   476.149994   467.206207   
2023-03-08   475.000000   479.350006   470.250000   471.450012   462.594513   
2023-03-09   471.500000   475.399994   467.000000   468.700012   459.896149   
...                 ...          ...          ...          ...          ...   
2024-07-08  1164.900024  1184.150024  1154.449951  1157.800049  1157.800049   
2024-07-09  1159.000000  1177.000000  1156.199951  1165.449951  1165.449951   
2024-07-10  1166.650024  1190.000000  1134.000000  1180.150024  1180.150024   
2024-07-11  1203.000000  1203.000000  1155.199951  1163.349976  1163.349976   
2024-07-12  1171.000000  1183.949951  1163.400024  1